# Load unique tweet tokens from file and remove links from tweets

### Save in another file the number of links for that tweet and the links list

In [386]:
import time
import numpy as np
import pandas as pd
from TokenizerWrapper import special_tokens, TokenizerWrapper

### Constants

In [387]:
N_ROWS = 100
HTTPS_TOKEN = '14120'
LINK_STRING = "https://t.co/"
LINK_TOKENS = '14120\t131\t120\t120\t188\t119\t11170\t120'

### Paths

In [388]:
TWEET_ID = "tweet_features_tweet_id"
TWEET_TOKENS = "tweet_features_text_tokens"

TWEET_TOKENS_FILE = "tweet_tokens/text_tokens_no_hashtags_mentions.csv"

RESULT_PATH = "tweet_tokens/text_tokens_clean.csv"
LINKS_PATH = "tweet_tokens/links/links.csv"

In [389]:
def get_remove_links(tokens):
    
    links_strings = []
    encoded_links = []
    
    if HTTPS_TOKEN in tokens_list:
        
        decoded_tokens = tok.decode(tokens).split(' ')

        mask = []

        length = len(decoded_tokens)

        finished  = False
        index = 0
        i = 0

        while i < length and not finished:

            dec_t = decoded_tokens[i]

            if dec_t == 'https':
                try:
                    index = i + 7
                    links_strings.append(decoded_tokens[index])
                    for j in range(8):
                        mask.append(False) # link splittato in 8 elementi tutti da rimuovere
                    i += 8
                    #print(initial_index, final_index)
                except:
                    #print(decoded_tokens)
                    #print(i)
                    #print(index)
                    #print(decoded_tokens[i])
                    for j in range(i, length):
                        mask.append(False)
                    finished = True
            else:
                mask.append(True)
                i += 1

        #print(decoded_tokens)
        #print(len(decoded_tokens))
        #print(mask)
        #print(len(mask))

        tokens_arr = np.array(decoded_tokens)
        tokens_arr = tokens_arr[mask]
        decoded_tokens = tokens_arr.tolist()
        tokens = tok.encode(' '.join(decoded_tokens))
        tokens = tokens[1:-1]  # ignore CLS ans SEP (they are duplicated)
    
        #print(tok.decode(tokens))
        # encode only the last string in each link (ignore the "https://t.co")
        for l in range(len(links_strings)):
            if links_strings[l] == '[SEP]':
                links_strings.pop(l)
            else:
                links_strings[l] = links_strings[l].replace(',', '').replace('.','')  # a link cointained ',' (in un csv spostava tutte le colonne)
                enc_l = tok.encode(links_strings[l]) 
                encoded_links.append(enc_l[1:-1])  # ignore CLS ans SEP
        
        #print(links_strings)
    
    else:
        tokens[-1] = '102'  # remove the last "\n" (tokens ends with "102\n")
            
    return tokens, encoded_links, links_strings

In [390]:

links_dict = {}
current_mapping = 0

def map_links(m_list):
    global links_dict, current_mapping
    mapped = []
    for m in m_list:
        if m not in links_dict:
            links_dict[m] = current_mapping
            current_mapping += 1
        
        mapped.append(links_dict[m])
    
    return mapped

In [391]:
def save_tweet(index, text_tokens):
    string = index + ',' + '\t'.join(map(str, text_tokens)) + '\n'
    result_file.write(string)
    

def save_links(text_tokens, text, mapped, count):
    for i in range(len(text_tokens)):
        text_tokens[i] = '\t'.join(map(str, text_tokens[i]))
    
    # each mentions is separated by a ";"
    # each token in a mention is 2008', '56210'separated by a "\t"
    string = str(count) + ',' + ';'.join(text_tokens) + ',' + '\t'.join(text) + ',' + '\t'.join(map(str, mapped)) + '\n'
    
    links_file.write(string)

In [392]:
def split_line(l):
    l = l.split(',')
    t_id = l[0]
    t_list = l[1].split('\t')  # replace("\\n",'').replace("\\t",'\t')
    
    return t_id, t_list

In [393]:
f_to_int = lambda x: int(x)
f_int = lambda x: list(map(f_to_int, x))

### Create a tokenizer

In [394]:
tok = TokenizerWrapper()

### Write the header (columns names)

In [395]:
result_file = open(RESULT_PATH, "w+")
links_file = open(LINKS_PATH, "w+")

In [396]:
result_file.write(TWEET_ID + ',' + TWEET_TOKENS + "\n")
links_file.write("links_count,links_tokens,links_text,links_mapping\n")

50

### Open files to be read

In [397]:
tokens_file = open(TWEET_TOKENS_FILE, "r")

### Execute

In [398]:
%%time

# ~3h EXECUTION

# ignore header
line = tokens_file.readline()

start = time.time()

finished = False
row = 0

while not finished:  # and row < 10000:
    
    links_tokens = []
    links_strings = []
    mapped_links = []
    links_count = 0
    
    if row % 1000000 == 0:
        elapsed_time = time.time() - start
        print('Row: ', row, ' - Elapsed time: ', elapsed_time)
            
    line = str(tokens_file.readline())
    
    #print(line)
    
    if line != '':
        
        tweet_id, tokens_list = split_line(line)

        #print('\ntweet_id: ', tweet_id)
        #print(tokens_list)
        #decoded_tweet = tok.decode(tokens_list)
        #print('\n', decoded_tweet, '\n')

        tokens_list, links_tokens, links_strings = get_remove_links(tokens_list)

        links_count = len(links_tokens)
        mapped_links = map_links(links_strings)

        #print('text tokens: ', tokens_list)
        #print('links: ', links_tokens)
        #print('links text: ', links_strings)
        #print('mapped links: ', mapped_links)
        #print('links count: ', links_count)

        save_tweet(tweet_id, tokens_list)
        save_links(links_tokens, links_strings, mapped_links, links_count)

    else:
        finished = True
            
    row += 1

Row:  0  - Elapsed time:  4.5299530029296875e-06
Row:  1000000  - Elapsed time:  424.5548131465912
Row:  2000000  - Elapsed time:  828.619425535202
Row:  3000000  - Elapsed time:  1208.9953210353851
Row:  4000000  - Elapsed time:  1584.635621547699
Row:  5000000  - Elapsed time:  1952.890278339386
Row:  6000000  - Elapsed time:  2317.298236846924
Row:  7000000  - Elapsed time:  2678.524081468582
Row:  8000000  - Elapsed time:  3033.8501262664795
Row:  9000000  - Elapsed time:  3384.5517551898956
Row:  10000000  - Elapsed time:  3734.719610929489
Row:  11000000  - Elapsed time:  4095.2562601566315
Row:  12000000  - Elapsed time:  4454.956626176834
Row:  13000000  - Elapsed time:  4808.66837477684
Row:  14000000  - Elapsed time:  5163.450683116913
Row:  15000000  - Elapsed time:  5511.863975524902
Row:  16000000  - Elapsed time:  5854.46142077446
Row:  17000000  - Elapsed time:  6193.926142454147
Row:  18000000  - Elapsed time:  6529.786390781403
Row:  19000000  - Elapsed time:  6865.178

In [399]:
tokens_file.close()

result_file.close()
links_file.close()

### Save mapping to json file

In [400]:
import json

In [401]:
json_links_mapping = json.dumps(links_dict)

In [402]:
with open('tweet_tokens/links/links_mapping.json', 'w+') as f:
    f.write(json_links_mapping)

In [403]:
len(links_dict)

18991547

### Check the dataset

In [382]:
%%time

df = pd.read_csv(RESULT_PATH,
                    #names=[TWEET_ID],
                    nrows=N_ROWS,
                    header=0,
                    index_col=0)

CPU times: user 8.26 ms, sys: 21 µs, total: 8.28 ms
Wall time: 6.26 ms


In [383]:
df

,tweet_features_text_tokens
tweet_features_tweet_id,
0,101\t6417\t3410\t3398\t3184\t1909\t56910\t1683...
1,101\t102
2,101\t62342\t10858\t54439\t19571\t22480\t7831\t...
3,101\t58955\t10898\t103305\t1901\t16181\t7168\t...
4,101\t2435\t5656\t2594\t8279\t8623\t1925\t64126...
...,...
95,101\t187\t14394\t12011\t102
96,101\t115\t18296\t33989\t54006\t10230\t14168\t2...
97,101\t123\t118\t10348\t118\t10197\t102


In [384]:
%%time

df = pd.read_csv(LINKS_PATH,
                    #names=[TWEET_ID],
                    nrows=N_ROWS,
                    header=0)

CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 9.56 ms


In [385]:
df

,links_tokens,links_text,links_mapping
links_count,,,
0,NaN,NaN,NaN
1,178\t71492\t11274\t10112\t10759\t11274\t11166\...,jbcBe1B5lP,0
1,177\t11779\t10418\t10457\t11274\t10874\t11733\...,iVjbBwKXHc,1
1,141\t10305\t25743\t13034\t10729\t11527\t10410\...,DzfsW2ttpL,2
0,NaN,NaN,NaN
...,...,...,...
1,172\t10738\t15417\t11779\t11011\t11537\t11166\...,dAJV4N5H2A,59
0,NaN,NaN,NaN
0,NaN,NaN,NaN
